In [1]:
1 + 1

2

In [ ]:
# スピン値(配列で表示)
σ = [-1, 1]

# ハミルトニアンの計算（隣接ペアの和）
function energy(config, L, J)　#config:スピン配置を1次元配列として渡す
    E = 0.0 #全体のエネルギーを初期化
    for x in 1:L
        for y in 1:L
            s = config[(x-1)*L + y]
            # 右隣（周期境界なし）
            if y < L
                s_r = config[(x-1)*L + y + 1]
                E -= J * s * s_r
            end
            # 下隣
            if x < L
                s_d = config[x*L + y]
                E -= J * s * s_d
            end
        end
    end
    return E
end

# 分配関数を計算
function partition_function(L; β=1.0, J=1.0)
    Z = 0.0
    N = L * L
    # 全スピン状態列挙（2^(L^2)）
    for i in 0:(2^N - 1)
        config = zeros(Int, N)
        for k in 1:N
            # ビットを使ってスピン割り当て
            config[k] = σ[bitstring(i)[end-k+1] == '1' ? 2 : 1]
        end
        E = energy(config, L, J)
        Z += exp(-β * E)
    end
    return Z
end

# 例：L=2, β=0.5, J=1.0
println("Z =", partition_function(2, β=0.5, J=1.0))

Z =27.048782764334526


In [9]:
#4脚テンソル
s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")
s3 = Index(2, "s3,c")
s4 = Index(2, "s4,d")

@show T4 = ITensor(s1, s2, s3, s4) #ここで4脚テンソルの関数を作成

T4 = ITensor(s1, s2, s3, s4) = ITensor ord=4
Dim 1: (dim=2|id=853|"a,s1")
Dim 2: (dim=2|id=792|"b,s2")
Dim 3: (dim=2|id=281|"c,s3")
Dim 4: (dim=2|id=531|"d,s4")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2×2×2




ITensor ord=4 (dim=2|id=853|"a,s1") (dim=2|id=792|"b,s2") (dim=2|id=281|"c,s3") (dim=2|id=531|"d,s4")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [10]:
#3脚テンソル
s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")
s3 = Index(2, "s3,c")

@show T3 = ITensor(s1, s2, s3)

T3 = ITensor(s1, s2, s3) = ITensor ord=3
Dim 1: (dim=2|id=324|"a,s1")
Dim 2: (dim=2|id=535|"b,s2")
Dim 3: (dim=2|id=600|"c,s3")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2×2




ITensor ord=3 (dim=2|id=324|"a,s1") (dim=2|id=535|"b,s2") (dim=2|id=600|"c,s3")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [11]:
#2脚テンソル
s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")

@show T2 = ITensor(s1, s2)

T2 = ITensor(s1, s2) = ITensor ord=2
Dim 1: (dim=2|id=695|"a,s1")
Dim 2: (dim=2|id=21|"b,s2")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2




ITensor ord=2 (dim=2|id=695|"a,s1") (dim=2|id=21|"b,s2")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [12]:
#1D3サイトの分配関数
Z = T3[s1=0,s2=1,s3=0] * T3[s1=0,s2=1,s3=0] 


MethodError: MethodError: no method matching getindex(::ITensor; s1::Int64, s2::Int64, s3::Int64)
This method may not support any kwargs.

Closest candidates are:
  getindex(::ITensor) got unsupported keyword arguments "s1", "s2", "s3"
   @ ITensors ~/.julia/packages/ITensors/elsyT/src/itensor.jl:1130
  getindex(::ITensor, Integer...) got unsupported keyword arguments "s1", "s2", "s3"
   @ ITensors ~/.julia/packages/ITensors/elsyT/src/itensor.jl:1087
  getindex(::ITensor, Union{Integer, ITensors.LastVal}...) got unsupported keyword arguments "s1", "s2", "s3"
   @ ITensors ~/.julia/packages/ITensors/elsyT/src/itensor.jl:1097
  ...
